# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [13]:
%pip install sentence-transformers
%pip install faiss-cpu
%pip install dotenv
%pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ---------------------------------------- 0.0/785.8 kB ? eta -:--:--
   ---------------------------------------- 785.8/785.8 kB 4.9 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 10.2 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import os
import faiss
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from dotenv import load_dotenv
from openai import OpenAI

## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [3]:
K = 25

In [4]:
df_news = pd.read_csv('../02-semantic-search-and-vector-database/df_news.csv')
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [5]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.
 

In [7]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'], 
        'TICKER': row['TICKER'], 
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [8]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [9]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        if metadata_filter:
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            D, I = temp_index.search(query_embedding, k)
            indices = [filtered_indices[i] for i in I[0]]
        else:
            D, I = self.index.search(query_embedding, k)
            indices = I[0]
            D = D[0]

        results = []
        for idx, sim in zip(indices, D):
            results.append((self.documents[idx], self.metadata[idx], float(sim)))
        return results

In [10]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [15]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
open_ai = OpenAI(api_key=api_key)

In [16]:
response = open_ai.responses.create(
      model="gpt-4o-mini",
      input="Write a short description of what a retrieval-augmented generation (RAG) system is.",
  )
print(response.output_text)

A Retrieval-Augmented Generation (RAG) system combines the strengths of information retrieval and natural language generation. It operates by first retrieving relevant documents or snippets from a large database based on a query. Once the relevant information is retrieved, the system uses a generative model to synthesize and produce coherent, contextually appropriate responses or summaries. This approach enhances the quality of generated content by grounding it in accurate, up-to-date information, making RAG systems particularly effective for tasks that require both knowledge retrieval and natural language understanding, such as answering questions or generating informative text.


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


In [ ]:
tickers = df_news['TICKER'].unique()
meta_data = []

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        meta_data.append({
            'TICKER': ticker,
            'COMPANY_NAME': info.get('longName', ''),
            'SECTOR': info.get('sector', ''),
            'INDUSTRY': info.get('industry', '')
        })
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")

df_meta = pd.DataFrame(meta_data)

In [19]:
df_meta.head(10)

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services
5,ADBE,Adobe Inc.,Technology,Software - Application
6,AMD,"Advanced Micro Devices, Inc.",Technology,Semiconductors
7,AES,The AES Corporation,Utilities,Utilities - Diversified
8,AFL,Aflac Incorporated,Financial Services,Insurance - Life
9,A,"Agilent Technologies, Inc.",Healthcare,Diagnostics & Research


## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [20]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

#### Predefined questions

In [21]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

In [55]:
# Function to avoid news duplication in the prompt
def group_by_document(results):
    grouped_results = {}

    for result in results:
        doc, meta, sim = result
        ticker = meta['TICKER']
        if doc not in grouped_results:
            grouped_results[doc] = []
        grouped_results[doc].append(ticker)

    return grouped_results

# Function to get the companies' info by their tickers
def get_companies_context(tickers):
    companies_context = []

    for ticker in tickers:
        company = df_meta[df_meta['TICKER'] == ticker]
        context = f"**{company['COMPANY_NAME'].values[0]}** which is part of the \"{company['INDUSTRY'].values[0]}\" industry and operates in the \"{company['SECTOR'].values[0]}\" sector"
        companies_context.append(context)

    return companies_context

# Function to build the prompt
def get_prompt(question):
    results = faiss_store.search(question)
    grouped_results = group_by_document(results)
    docs_and_companies = []

    for doc, tickers in grouped_results.items():
        companies = ', and '.join(get_companies_context(tickers))
        docs_and_companies.append(f"* \"{doc}\" that involves companies such as {companies}.\n")
    
    prompt = f"As an expert in financial analysis and deep understanding of the financial markets, tell me **{question}** based on the following news headers: \n {''.join(docs_and_companies)}"
    
    return prompt

def get_open_ai_responses(questions):
    for question in questions:
        prompt = get_prompt(question)
        response = open_ai.responses.create(
            model="gpt-4o-mini",
            input=prompt
        )
        print_markdown(f"# **QUESTION**: \n {question} \n\n")
        print_markdown(f"# **PROMPT** \n {prompt} \n\n")
        print_markdown(f"# **OPEN AI'S ANSWER** \n {response.output_text} \n\n")
        print_markdown("---------------------------------------------------------------------- \n\n\n")

In [56]:
get_open_ai_responses(questions_topic)
get_open_ai_responses(questions_company)
get_open_ai_responses(questions_industry)

# **QUESTION**: 
 What are the major concerns expressed in financial news about inflation? 



# **PROMPT** 
 As an expert in financial analysis and deep understanding of the financial markets, tell me **What are the major concerns expressed in financial news about inflation?** based on the following news headers: 
 * "Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown." that involves companies such as **BlackRock, Inc.** which is part of the "Asset Management" industry and operates in the "Financial Services" sector.
* "The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead" that involves companies such as **Tesla, Inc.** which is part of the "Auto Manufacturers" industry and operates in the "Consumer Cyclical" sector, and **NVIDIA Corporation** which is part of the "Semiconductors" industry and operates in the "Technology" sector, and **lululemon athletica inc.** which is part of the "Apparel Retail" industry and operates in the "Consumer Cyclical" sector, and **Broadcom Inc.** which is part of the "Semiconductors" industry and operates in the "Technology" sector.
 



# **OPEN AI'S ANSWER** 
 The concerns expressed in financial news about inflation, based on the provided headlines, highlight several key issues affecting various sectors and companies:

### 1. **Persistent Inflation Risks**
   - **Federal Reserve Concerns**: The Fed’s minutes indicate a growing worry about persistent inflation. This suggests that inflation is not just a temporary spike but may be entrenched, requiring more aggressive monetary policy measures that could impact economic growth. The implications for asset management firms like **BlackRock** are significant, as their strategies may need to adapt to a shifting interest rate environment which is influenced by inflationary pressures.

### 2. **Impact on Interest Rates**
   - **Rate Cuts Off the Table**: The expectation of food inflation dampening hopes for a rate cut signals that the Fed may be hesitant to lower interest rates, which would typically provide more liquidity to the markets. This affects companies across sectors, including **Tesla** and **NVIDIA**, by potentially increasing borrowing costs and reducing consumer spending power.

### 3. **Sector-Specific Challenges**
   - **Consumer Cyclical Sector**: Companies like **Tesla**, **lululemon**, and others experience direct impacts from rising inflation in terms of consumer demand. Higher prices can lead to lower disposable income, negatively affecting sales. If inflation affects food and essential items, consumers may prioritize these over discretionary spending on cars or luxury apparel.
   - **Technology Sector**: For companies such as **NVIDIA** and **Broadcom**, persistent inflation may increase the cost of raw materials and components, squeezing profit margins and potentially slowing down technological innovation if companies face budget constraints.

### 4. **Global Supply Chain Issues**
   - **Tariff and Trade Dynamics**: Rising tariffs and trade tensions can exacerbate inflation, as companies may face increased costs that are often passed on to consumers. This creates a vicious cycle where elevated prices lead to decreased consumer purchasing power and further economic challenges.

### 5. **Market Sentiment and Volatility**
   - **Market Reactions**: The slip in Bitcoin prices coinciding with concerns over inflation reflects a broader anxiety in the market. Investors often react to inflation news by reallocating their portfolios, resulting in heightened volatility, particularly in speculative assets like cryptocurrencies.

### 6. **Broader Economic Implications**
   - **Economic Slowdown Risks**: The combination of persistent inflation and potential monetary tightening raises fears of an economic slowdown. If growth falters, this could impact corporate earnings across sectors, highlighting the interconnectedness of inflation with overall market performance.

### Summary
Overall, these headlines reflect a multifaceted view of inflation concerns influencing monetary policy, consumer behavior, and corporate profitability. Each sector, from asset management to consumer cyclical and technology, faces unique challenges arising from a sustained inflation environment, posing risks for future economic growth. 



---------------------------------------------------------------------- 




# **QUESTION**: 
 How is investor sentiment described in recent financial headlines? 



...

---------------------------------------------------------------------- 




# **QUESTION**: 
 What risks or challenges are discussed in recent news about the energy industry? 



# **PROMPT** 
 As an expert in financial analysis and deep understanding of the financial markets, tell me **What risks or challenges are discussed in recent news about the energy industry?** based on the following news headers: 
 * "Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law." that involves companies such as **NextEra Energy, Inc.** which is part of the "Utilities - Regulated Electric" industry and operates in the "Utilities" sector, and **Enphase Energy, Inc.** which is part of the "Solar" industry and operates in the "Technology" sector.
* "Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?" that involves companies such as **Halliburton Company** which is part of the "Oil & Gas Equipment & Services" industry and operates in the "Energy" sector, and **Baker Hughes Company** which is part of the "Oil & Gas Equipment & Services" industry and operates in the "Energy" sector.
* "3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty." that involves companies such as **Freeport-McMoRan Inc.** which is part of the "Copper" industry and operates in the "Basic Materials" sector.
 



# **OPEN AI'S ANSWER** 
 In recent news, several critical risks and challenges have emerged in the energy industry, particularly affecting renewable energy, oilfield services, and the broader implications of tariff policies. Here’s a breakdown based on the provided articles:

### Renewable Energy Industry Challenges
1. **Legislative Risks**:
   - The **U.S. Congress** is advancing a bill that could repeal significant subsidies for the renewable energy sector, threatening the economic viability of renewable projects. Companies like **NextEra Energy** and **Enphase Energy** could see their investments become uneconomical if incentives are reduced or eliminated.

2. **Market Volatility**:
   - The potential loss of subsidies may create volatility in renewable energy stocks, impacting investor confidence and leading to substantial stock declines, as seen in the recent crash.

3. **Increased Production Costs**:
   - As companies have ramped up production in anticipation of continued support, the abrupt changes in policy could lead to surplus production capacity, pushing down prices and squeezing margins.

### Oilfield Services Sector Risks
1. **Oil Price Volatility**:
   - Declining oil prices are creating uncertainty for companies like **Halliburton** and **Baker Hughes**, as lower commodity prices typically lead to reduced drilling budgets and investments in oilfield services.

2. **Rising Tariffs**:
   - Increasing tariffs on imported materials and equipment can drive up operational costs for oilfield service companies, further straining margins and competitiveness.

3. **Shifting Demand**:
   - While there is potential support from growing demand for Liquefied Natural Gas (LNG) and applications of AI in oil extraction, there remains uncertainty about the sustainability of these markets amidst broader decreases in drilling activity.

### Tariff Policies Impacting Basic Materials
1. **Trade Uncertainty**:
   - The evolving tariff landscape under the previous U.S. administration poses risks for companies like **Freeport-McMoRan**. Unpredictability regarding tariffs can disrupt cost structures and pricing in the copper industry.

2. **Supply Chain Disruptions**:
   - Loopholes and inconsistent tariff implementations can lead to supply chain complications, affecting production efficiency and cost competitiveness in the broader industrial economy.

3. **Market Competition**:
   - Any improvements in tariff policies aimed at better positioning for American creators could lead to increased competition, where companies must rapidly adapt to changing economic conditions and consumer demand.

### General Industry Observations
- **Regulatory Environment**: Both sectors are increasingly sensitive to changes in government policies and regulations, which can swiftly affect operational viability and financial health.
- **Global Market Dynamics**: As global transitions towards renewable sources continue, both sectors must navigate these shifts while addressing underlying market demand and logistical complexities.

In summary, companies in the energy domain face numerous challenges, including legislative risks that threaten financial incentives, volatile market conditions arising from fluctuating oil prices, and the complexities of navigating an uncertain tariff landscape. 



---------------------------------------------------------------------- 




## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance** 
2. **Completeness**  
3. **Bias or Noise** 
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

YOUR WRITTEN RESPONSE HERE


#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

YOUR WRITTEN RESPONSE HERE
  

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

YOUR WRITTEN RESPONSE HERE


#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

YOUR WRITTEN RESPONSE HERE
  

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

YOUR WRITTEN RESPONSE HERE
  

## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [ ]:
# CODE HERE
# Use as many coding cells as you need

## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



YOUR WRITTEN RESPONSE HERE